### Import Lib

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
!pip install neupy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
import re
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import KFold
from sklearn.metrics import hamming_loss,classification_report,f1_score,zero_one_loss
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import VotingRegressor

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 226 kB 8.6 MB/s 
     |████████████████████████████████| 92.7 MB 1.1 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 367 kB 71.7 MB/s 
     |████████████████████████████████| 3.2 MB 50.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

### Read Dataset

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1wCpUu5mCfVUHVTmMBo7sqTax5F7EDfrW' -O datasetalquran.xlsx 

--2022-01-25 02:08:09--  https://drive.google.com/uc?export=download&id=1wCpUu5mCfVUHVTmMBo7sqTax5F7EDfrW
Resolving drive.google.com (drive.google.com)... 74.125.195.113, 74.125.195.138, 74.125.195.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.195.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-04-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/dtnib5sdm9n79lpfbjveht6o8mmv053f/1643076450000/16601388677960215569/*/1wCpUu5mCfVUHVTmMBo7sqTax5F7EDfrW?e=download [following]
--2022-01-25 02:08:11--  https://doc-00-04-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/dtnib5sdm9n79lpfbjveht6o8mmv053f/1643076450000/16601388677960215569/*/1wCpUu5mCfVUHVTmMBo7sqTax5F7EDfrW?e=download
Resolving doc-00-04-docs.googleusercontent.com (doc-00-04-docs.googleusercontent.com)... 74.125.142.132, 2607:f8b0:400e:c08::84
Connecting to doc-00-04-docs.googleusercontent.com (doc-00

In [ ]:
df = pd.read_excel('datasetalquran.xlsx')

### Preprocessing

In [ ]:
kelas={}
#case folding
df['Terjemahan']=df['Terjemahan'].apply(lambda x: x.lower())
#punctual removal
df['Terjemahan']=df['Terjemahan'].apply(lambda x: re.sub('[^a-zA-z\s]','',x))
#stemming
ps=PorterStemmer()
df['Terjemahan']=df['Terjemahan'].apply(lambda x: " ".join([ps.stem(y) for y in x.split(" ")]))      
#stopword removal
stop_word= set (stopwords.words("english"))
stop_word.remove('did') 
df['Terjemahan']=df['Terjemahan'].apply(lambda x: " ".join([item for item in x.split(" ") if item not in stop_word]))

In [ ]:
#inisiasi
k=0;
tohammingloss=[]

model = GaussianNB()
model2 = MultinomialNB()
model3 = ComplementNB()
model4 = BernoulliNB()
model5 = RandomForestClassifier(n_estimators=10)
model6 = tree.DecisionTreeClassifier()
model7 = GradientBoostingClassifier(n_estimators=10)
model8 = VotingClassifier(estimators=[('mnb', model2), ('cnb', model2), ('bnb', model2)],voting='soft')
model9 = VotingRegressor(estimators=[('mnb', model2), ('cnb', model2), ('bnb', model2)])

In [ ]:
hm=[]
kf=KFold(n_splits=5)#inisiasi kfold
for train, test in kf.split(df):#jadi 5 train dan 5 test
    allpredik=pd.DataFrame()#inislaisai
    train_data = np.array(df)[train]#simpan index data
    test_data = np.array(df)[test]#simpan index data
    X_train=pd.Series(np.resize(train_data[:,[3]],(train_data[:,[0]].size,)), dtype='str')#ambil data ayat
    X_test=pd.Series(np.resize(test_data[:,[3]],(test_data[:,[0]].size,)), dtype='str')#ambil data ayat
    allytest=test_data[:,[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]]#ambil data label 
    tokenizer = Tokenizer( split=' ')
    tokenizer.fit_on_texts(X_train)#ambil tiap kata
    X_train1 = tokenizer.texts_to_matrix(X_train, mode='tfidf') #tokenisasi dan TFIDF
    X_test=tokenizer.texts_to_matrix(X_test, mode='tfidf') #tokenisasi dan TFIDF
    for kelas in range(15):       
      Y_train=train_data[:,[kelas+4]]
      Y_test=test_data[:,[kelas+4]]
      # X_train, Y_train = resample(X_train1, Y_train.ravel().astype(int))
      # if skenariosampling=="smote":
      X_train=X_train1
      Y_train=Y_train.ravel().astype(int)
      # print(X_train.shape,Y_train.shape)
      # print(X_test.shape,Y_test.shape)
      gnb = model.fit(X_train, Y_train)
      # mnb = model2.fit(X_train, Y_train)
      # cnb = model3.fit(X_train, Y_train)
      # bnb = model4.fit(X_train, Y_train)
      # lr = model5.fit(X_train, Y_train)
      # svm = model6.fit(X_train, Y_train)
      # rf = model7.fit(X_train, Y_train)
      # vc = model8.fit(X_train, Y_train)
      # vr = model9.fit(X_train, Y_train)
      predik1 = gnb.predict(X_test)
      # predik2 = mnb.predict(X_test)
      # predik3 = cnb.predict(X_test)
      # predik4 = bnb.predict(X_test)
      # predik5 = lr.predict(X_test)
      # predik6 = svm.predict(X_test)
      # predik7 = rf.predict(X_test)
      # predik8 = vc.predict(X_test)
      # predik9 = vr.predict(X_test)
      tablepred = []#simpan predik 1 kelas
      for i in range (0,len(X_test)):
        # if sum([predik2[i],predik3[i],predik4[i]]) == 3:
        if sum([predik1[i]]) == 1:
        # if sum([predik3[i],predik4[i]]) > 1:
        # if sum([predik3[i],predik1[i],predik4[i]]) > 1:
        # if sum([predik1[i],predik2[i],predik3[i],predik4[i]]) > 2:
          tablepred.append(1)
        else:
          tablepred.append(0)
      allpredik[str(kelas+1)]=tablepred
    #predik jadi array
    p=[]
    for pr  in np.array(allpredik):
      if sum(pr)==0:
        p.append(np.append(pr,1).tolist())
      else:
        p.append(np.append(pr,0).tolist())
    # hasil prediksi
    predik=np.asarray(p,dtype="int")
    #  menghitung hamming loss
    print("Hammming loss: %.4f" % (hamming_loss(allytest.astype(dtype='int32'),predik)))
    #  menghitung hamming loss
    hm.append(hamming_loss(allytest.astype(dtype='int32'),predik))
    print(hm)

Hammming loss: 0.2451
[0.24514222756410256]
Hammming loss: 0.2783
[0.24514222756410256, 0.2783179631114675]
Hammming loss: 0.3479
[0.24514222756410256, 0.2783179631114675, 0.3479350441058541]
Hammming loss: 0.3592
[0.24514222756410256, 0.2783179631114675, 0.3479350441058541, 0.3592121090617482]
Hammming loss: 0.3908
[0.24514222756410256, 0.2783179631114675, 0.3479350441058541, 0.3592121090617482, 0.3907878909382518]


In [ ]:
np.mean(hm)

0.32427904695628484